In [1]:
from py_pdf_parser.loaders import load_file

document = load_file("../Malo_Adler_Thesis.pdf")
text2 = ""
for element in document.elements:
    text2 += element.text()

from textrank import Model
embedding_model = Model.from_hugging_hub("dangvantuan/sentence-camembert-base")

No elements detected on page 2, skipping this page.
/home/jerome/miniforge3/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from summarize_chain import summarize_chain_builder

custom_chain = summarize_chain_builder(method="refine")
res = custom_chain.invoke(text2[:10_000])
res

'The text discusses various methods for evaluating the similarity between a prediction and a reference in summarization tasks, including ROUGE, chrF++, MoverScore, BERTScore, and Grouped-Query Attention (GQA). The author also mentions their use of K-Means clustering algorithm to preserve semantic diversity and their implementation of the TextRank algorithm for extractive summarization. Additionally, they mention using pre-trained language models such as Llama and comparing the BERTScore between samples generated by the model and a double-checker. The author also mentions that time is an important factor in their specific use case and that TextRank is a more suitable choice for long source texts.'